# Recommend Products using SparkML

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

As for the notebooks, to run these you will need to register for a free Databricks
[Community Edition account](https://community.cloud.databricks.com/)

## import modules

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

## Design Schema

In [5]:
rate_schema = "`userid` string, `accoid` string, `rating` INT"
accos_schema = "`id` string, `title` string, `location` string, `price` int, `rooms` int, `rating` float"

## Read csv files to Spark DataFrame

In [7]:
rating_file_location = '/FileStore/tables/rating-1.csv'
dfRates = spark.read.csv(rating_file_location, rate_schema)

print(dfRates.count())
display(dfRates)

userid,accoid,rating
10,1,1
18,1,2
13,1,1
7,2,2
4,2,2
13,2,3
19,2,2
12,2,1
11,2,1
1,2,2


In [8]:
print(dfRates.count())

1187

In [9]:
accos_file_location = "/FileStore/tables/accommodation-1.csv"
dfAccos = spark.read.csv(accos_file_location, accos_schema)

print(dfAccos.count())
display(dfAccos)

id,title,location,price,rooms,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.1
2,Cozy Calm Hut,London,65,2,4.1
3,Agreable Calm Place,London,65,4,4.8
4,Colossal Quiet Chateau,Paris,3400,16,2.7
5,Homy Quiet Shack,Paris,50,1,1.1
6,Pleasant Quiet Place,Dublin,35,5,4.3
7,Vast Peaceful Fortress,Seattle,3200,24,1.9
8,Giant Quiet Fortress,San Francisco,3400,12,4.1
9,Giant Peaceful Palace,London,1500,20,3.5
10,Sizable Calm Country House,Auckland,650,9,4.9


## Aggregations

In [11]:
display(dfRates)

userid,accoid,rating
10,1,1
18,1,2
13,1,1
7,2,2
4,2,2
13,2,3
19,2,2
12,2,1
11,2,1
1,2,2


In [12]:
df_agg = dfRates.agg(F.count('userid').alias('num_ratings'),
            F.countDistinct('userid').alias('distinct_users_rating'),
            F.max('rating').alias('best_rating'),
            F.min('rating').alias('worst_rating'),
            F.avg('rating').alias('avg_rating')
           )
display(df_agg)

num_ratings,distinct_users_rating,best_rating,worst_rating,avg_rating
1187,25,5,1,2.4667228306655433


## Left Join

In [14]:
df_leftjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'left')
print(df_leftjoined.count())
display(df_leftjoined)

id,title,location,price,rooms,ratings,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,13,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,10,1,1
2,Cozy Calm Hut,London,65,2,4.1,3,2,1
2,Cozy Calm Hut,London,65,2,4.1,2,2,4
2,Cozy Calm Hut,London,65,2,4.1,20,2,2
2,Cozy Calm Hut,London,65,2,4.1,1,2,2
2,Cozy Calm Hut,London,65,2,4.1,11,2,1
2,Cozy Calm Hut,London,65,2,4.1,12,2,1
2,Cozy Calm Hut,London,65,2,4.1,19,2,2


In [15]:
display(df_leftjoined.select('*').where(df_leftjoined.id == '100'))

id,title,location,price,rooms,ratings,userid,accoid,rating
100,Villa Staden,Stockholm,80000,4,4.2,null,null,null


## Right Join

In [17]:
df_rightjoined = dfAccos.withColumnRenamed('rating', 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'right')
print(df_rightjoined.count())
display(df_rightjoined)

id,title,location,price,rooms,ratings,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,13,1,1
2,Cozy Calm Hut,London,65,2,4.1,7,2,2
2,Cozy Calm Hut,London,65,2,4.1,4,2,2
2,Cozy Calm Hut,London,65,2,4.1,13,2,3
2,Cozy Calm Hut,London,65,2,4.1,19,2,2
2,Cozy Calm Hut,London,65,2,4.1,12,2,1
2,Cozy Calm Hut,London,65,2,4.1,11,2,1
2,Cozy Calm Hut,London,65,2,4.1,1,2,2


In [18]:
df_rightjoined.select('*').where(df_rightjoined.id == '100').show()

+---+-----+--------+-----+-----+-------+------+------+------+
 id|title|location|price|rooms|ratings|userid|accoid|rating|
+---+-----+--------+-----+-----+-------+------+------+------+
+---+-----+--------+-----+-----+-------+------+------+------+

In [19]:
display(df_rightjoined.select('*').where(df_rightjoined.accoid == '101'))

id,title,location,price,rooms,rating,userid,accoid,rating
null,null,null,null,null,null,23,101,5


## Inner Join

In [21]:
df_innerjoined = dfAccos.withColumnRenamed('rating' , 'ratings').join(dfRates, dfAccos.id == dfRates.accoid, how = 'inner')
print(df_innerjoined.count())
display(df_innerjoined)

id,title,location,price,rooms,ratings,userid,accoid,rating
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,10,1,1
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,18,1,2
1,Comfy Quiet Chalet,Vancouver,50,3,3.1,13,1,1
2,Cozy Calm Hut,London,65,2,4.1,7,2,2
2,Cozy Calm Hut,London,65,2,4.1,4,2,2
2,Cozy Calm Hut,London,65,2,4.1,13,2,3
2,Cozy Calm Hut,London,65,2,4.1,19,2,2
2,Cozy Calm Hut,London,65,2,4.1,12,2,1
2,Cozy Calm Hut,London,65,2,4.1,11,2,1
2,Cozy Calm Hut,London,65,2,4.1,1,2,2


In [22]:
df_innerjoined.select('*').where(df_innerjoined.id == '100').show()

+---+-----+--------+-----+-----+-------+------+------+------+
 id|title|location|price|rooms|ratings|userid|accoid|rating|
+---+-----+--------+-----+-----+-------+------+------+------+
+---+-----+--------+-----+-----+-------+------+------+------+

In [23]:
df_innerjoined.select('*').where(df_joined.accoid == '101').show()

+---+-----+--------+-----+-----+------+------+------+------+
 id|title|location|price|rooms|rating|userid|accoid|rating|
+---+-----+--------+-----+-----+------+------+------+------+
+---+-----+--------+-----+-----+------+------+------+------+

## Train the model and recommend products with the model

### Train the model

In [26]:
model = ALS.train(dfRates.rdd, 20, 20)
print(type(model))

<class 'pyspark.mllib.recommendation.MatrixFactorizationModel'>

### Save the model

In [28]:
model.save(sc,'/FileStore/model_recommendation.ml' )

### Use the trained model to predict what accommodations each user might be interested

In [30]:
allPredictions = None
for USER_ID in range(0, 100):
  dfUserRatings = dfRates.filter(dfRates.userid == USER_ID).rdd.map(lambda r: r.accoid).collect()
  rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
  pairsPotential = rddPotential.map(lambda x: (USER_ID, x[0]))
  predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))
  predictions = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5
  print("predicted for user={0}".format(USER_ID))
  if (allPredictions == None):
    allPredictions = predictions
  else:
    allPredictions.extend(predictions)

predicted for user=0
predicted for user=1
predicted for user=2
predicted for user=3
predicted for user=4
predicted for user=5
predicted for user=6
predicted for user=7
predicted for user=8
predicted for user=9
predicted for user=10
predicted for user=11
predicted for user=12
predicted for user=13
predicted for user=14
predicted for user=15
predicted for user=16
predicted for user=17
predicted for user=18
predicted for user=19
predicted for user=20
predicted for user=21
predicted for user=22
predicted for user=23
predicted for user=24
predicted for user=25
predicted for user=26
predicted for user=27
predicted for user=28
predicted for user=29
predicted for user=30
predicted for user=31
predicted for user=32
predicted for user=33
predicted for user=34
predicted for user=35
predicted for user=36
predicted for user=37
predicted for user=38
predicted for user=39
predicted for user=40
predicted for user=41
predicted for user=42
predicted for user=43
predicted for user=44
predicted for user=45
predicted for user=46
predicted for user=47
predicted for user=48
predicted for user=49
predicted for user=50
predicted for user=51
predicted for user=52
predicted for user=53
predicted for user=54
predicted for user=55
predicted for user=56
predicted for user=57
predicted for user=58
predicted for user=59
predicted for user=60
predicted for user=61
predicted for user=62
predicted for user=63
predicted for user=64
predicted for user=65
predicted for user=66
predicted for user=67
predicted for user=68
predicted for user=69
predicted for user=70
predicted for user=71
predicted for user=72
predicted for user=73
predicted for user=74
predicted for user=75
predicted for user=76
predicted for user=77
predicted for user=78
predicted for user=79
predicted for user=80
predicted for user=81
predicted for user=82
predicted for user=83
predicted for user=84
predicted for user=85
predicted for user=86
predicted for user=87
predicted for user=88
predicted for user=89
predicted for user=90
predicted for user=91
predicted for user=92
predicted for user=93
predicted for user=94
predicted for user=95
predicted for user=96
predicted for user=97
predicted for user=98
predicted for user=99

In [31]:
type(allPredictions)

Out[35]: list

In [32]:
allPredictions

Out[36]: [('0', '76', 3.403691443278325),
 ('0', '75', 3.336812472784745),
 ('0', '66', 3.3008071142769984),
 ('0', '49', 3.2894170910892155),
 ('0', '39', 3.263942726449738),
 ('1', '85', 2.704729349265073),
 ('1', '43', 2.2708598565121285),
 ('1', '18', 2.2280416289393328),
 ('1', '47', 1.7544994780396697),
 ('1', '16', 1.7196868248383317),
 ('2', '87', 3.1235827870859394),
 ('2', '57', 3.0375820924470553),
 ('2', '11', 2.7332912338907533),
 ('2', '92', 2.4842167909197688),
 ('2', '25', 2.273061531816794),
 ('3', '30', 4.566452613317836),
 ('3', '6', 4.36898084096784),
 ('3', '3', 4.212977812191545),
 ('3', '12', 4.199101340430862),
 ('3', '16', 3.7147303616487943),
 ('4', '7', 2.506063952539394),
 ('4', '18', 1.6538781080206433),
 ('4', '73', 1.6485007015726993),
 ('4', '80', 1.6450501611241863),
 ('4', '40', 1.6294106216186215),
 ('5', '98', 2.391421952772769),
 ('5', '12', 2.2518922935994192),
 ('5', '87', 2.19193220334263),
 ('5', '6', 2.120079836961427),
 ('5', '85', 2.1178758008817673),
 ('6', '30', 4.607154524202949),
 ('6', '3', 4.595127426976358),
 ('6', '72', 4.366918101668251),
 ('6', '59', 4.348729523178536),
 ('6', '76', 4.304487078077898),
 ('7', '34', 2.0918643435078677),
 ('7', '88', 1.7986258071191772),
 ('7', '54', 1.7700253239425372),
 ('7', '81', 1.5260283254303972),
 ('7', '77', 1.427768122338275),
 ('8', '67', 2.4249259891124426),
 ('8', '81', 1.9958549001142103),
 ('8', '46', 1.8630252522950448),
 ('8', '40', 1.861871741908459),
 ('8', '24', 1.7248626740583903),
 ('9', '30', 4.283842401246601),
 ('9', '6', 3.9564335272040445),
 ('9', '38', 3.749877614853589),
 ('9', '3', 3.635351823412824),
 ('9', '22', 3.522333536141247),
 ('10', '7', 1.999370907373831),
 ('10', '51', 1.8071385247343177),
 ('10', '21', 1.6337771156094365),
 ('10', '64', 1.594450696686755),
 ('10', '43', 1.588555862709298),
 ('11', '51', 2.656671225736389),
 ('11', '30', 2.2921480980388362),
 ('11', '88', 2.19591326915857),
 ('11', '12', 2.0749202259631403),
 ('11', '81', 1.9370266437529833),
 ('12', '30', 3.808813307924532),
 ('12', '38', 3.6951654785909698),
 ('12', '6', 3.527855699124146),
 ('12', '12', 3.3972114295487774),
 ('12', '16', 3.1662253601641632),
 ('13', '6', 4.136014287920674),
 ('13', '12', 3.962484869319531),
 ('13', '30', 3.960178737194383),
 ('13', '3', 3.611898225691541),
 ('13', '33', 3.596751984837689),
 ('14', '97', 2.4632000806632712),
 ('14', '86', 2.4383543733675914),
 ('14', '49', 2.4286916763173987),
 ('14', '33', 2.424329533563973),
 ('14', '85', 2.3520560031446562),
 ('15', '30', 3.8493934619283596),
 ('15', '6', 3.8390140986742285),
 ('15', '12', 3.5196246142720216),
 ('15', '22', 3.093409248955065),
 ('15', '38', 3.093229327197687),
 ('16', '81', 1.6556879952466685),
 ('16', '94', 1.5451730748258343),
 ('16', '80', 1.4898737660918822),
 ('16', '47', 1.2347269037467659),
 ('16', '45', 1.2005926911670293),
 ('17', '87', 3.6267052766901897),
 ('17', '95', 3.4264408826734387),
 ('17', '25', 2.6556283549355495),
 ('17', '46', 2.291556795843921),
 ('17', '57', 2.191174747246262),
 ('18', '12', 4.455266419701198),
 ('18', '30', 4.281365404269112),
 ('18', '6', 4.080756094052909),
 ('18', '3', 3.8682739685439302),
 ('18', '16', 3.763797097327159),
 ('19', '59', 3.402886904647172),
 ('19', '66', 3.1895232716045823),
 ('19', '30', 3.169694495098188),
 ('19', '72', 3.0687742308469677),
 ('19', '49', 3.046075991966686),
 ('20', '76', 3.5711831248427295),
 ('20', '86', 3.416947719723195),
 ('20', '61', 3.269373767160739),
 ('20', '66', 3.207436637230604),
 ('20', '72', 3.166858718404901),
 ('21', '13', 1.6119710506022644),
 ('21', '20', 1.3690349013916037),
 ('21', '69', 1.3642118959780571),
 ('21', '48', 1.285658036220117),
 ('21', '52', 1.2699300709230905),
 ('22', '30', 4.591980289076685),
 ('22', '6', 4.40937615960305),
 ('22', '38', 4.0003599565636),
 ('22', '12', 3.795494451952764),
 ('22', '59', 3.7872631062581394),
 ('23', '87', 1.9223081939451891),
 ('23', '71', 1.5858152140737434),
 ('23', '47', 1.53633783

### Prediction for user '1'

In [34]:
dfUserRatings = dfRates.filter(dfRates.userid == '1').rdd.map(lambda r: r.accoid).collect()
rddPotential  = dfAccos.rdd.filter(lambda x: x[0] not in dfUserRatings)
pairsPotential = rddPotential.map(lambda x: ('1', x[0]))

In [35]:
pairsPotential.collect()

Out[41]: [('1', '1'),
 ('1', '3'),
 ('1', '6'),
 ('1', '9'),
 ('1', '10'),
 ('1', '12'),
 ('1', '13'),
 ('1', '15'),
 ('1', '16'),
 ('1', '17'),
 ('1', '18'),
 ('1', '20'),
 ('1', '21'),
 ('1', '22'),
 ('1', '26'),
 ('1', '28'),
 ('1', '29'),
 ('1', '30'),
 ('1', '31'),
 ('1', '32'),
 ('1', '33'),
 ('1', '35'),
 ('1', '38'),
 ('1', '39'),
 ('1', '41'),
 ('1', '43'),
 ('1', '46'),
 ('1', '47'),
 ('1', '49'),
 ('1', '50'),
 ('1', '57'),
 ('1', '59'),
 ('1', '61'),
 ('1', '63'),
 ('1', '64'),
 ('1', '65'),
 ('1', '66'),
 ('1', '69'),
 ('1', '70'),
 ('1', '72'),
 ('1', '74'),
 ('1', '75'),
 ('1', '76'),
 ('1', '78'),
 ('1', '79'),
 ('1', '83'),
 ('1', '84'),
 ('1', '85'),
 ('1', '86'),
 ('1', '90'),
 ('1', '93'),
 ('1', '96'),
 ('1', '99'),
 ('1', '100')]

In [36]:
predictions = model.predictAll(pairsPotential).map(lambda p: (str(p[0]), str(p[1]), float(p[2])))

In [37]:
predictions = predictions.takeOrdered(5, key=lambda x: -x[2]) # top 5

#### These are the five accommodations that are recommended to user '1'. Note that the quality of the recommendations are not great because the dataset was so small (the predicted ratings are not very high). Still, this lab illustrates the process you'd go through to create product recommendations.

In [39]:
predictions

Out[46]: [('1', '85', 3.028747509074468),
 ('1', '43', 2.7110084537290664),
 ('1', '18', 1.9584766151851714),
 ('1', '47', 1.7778684117149512),
 ('1', '46', 1.4865633925562696)]